# Examen deel 1 : Data load : data science Syntra, Tim Hellemans, J. Van Erum, juni 2024

# A : nodige modules

In [19]:
# standaard in python
import os 
import datetime
import sys
import json
import random
import calendar
import gzip

In [20]:
# te installeren packages en modules
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

from scipy import stats
# from scipy.stats import weibull_min
# from scipy.stats import skewnorm
# from scipy.stats import pareto
# from scipy.stats import beta
from scipy.stats import norm

# from sklearn.linear_model import LinearRegression

from tqdm import tqdm

In [21]:
# eigen packages modules
current_working_directory = os.getcwd()                                    
parent_working_directory = os.path.dirname(current_working_directory)      
scripts_directory = os.path.join(parent_working_directory, 'scripts')      
sys.path.append(scripts_directory)                                         
# sys.path.remove(scripts_directory)
# print(sys.path)

import come_in_handy as cih
import model_simulatie as mdls
import model_data_loaders as mdll
import my_matplotlib_standards as mypltstd

In [22]:
# eigen standard set up uitvoering
mypltstd.my_matplotlib_pyplot_standards_setup()

# B : functies nog niet in modules

# C : opladen van de gegevens

In [23]:
# opladen JSON file met MSR waarden (min_sust_rate = minimu sustainable rate)
min_sust_rate_locatie = "../data/raw/productiemodel/data_productie/master_data.json"
df_msr_gegevens = mdll.laad_msr_gegevens_in_dataframe(min_sust_rate_locatie)
# df_msr_gegevens

In [15]:
# bepaal waar de dagelijk productie sets (per fabriek) zich bevinden op schijf
dagelijkse_productie_locatie = "../data/raw/productiemodel/data_productie/daily_production/"

In [16]:
# Laad dagelijks productie voor BRU in BRU dataframe
factory = 'BRU'
dagelijkse_productie_BRU = mdll.laad_dagelijkse_productie_in_dataframe(
                                factory, 
                                os.path.join(dagelijkse_productie_locatie, factory))
# dagelijkse_productie_BRU

# voeg MSR waarden toe op het STO dagelijkse productie dataframe
dagelijkse_productie_BRU = dagelijkse_productie_BRU.merge(
                                        df_msr_gegevens, on='factory', how='left')
# dagelijkse_productie_BRU

# verwijder records waar onderhoud werd gepland, zie opgave Deel 1, stap 5, bullet 1 en dixit Tim in de klas
dagelijkse_productie_BRU = dagelijkse_productie_BRU[dagelijkse_productie_BRU['maintenance'] == 'No']
# dagelijkse_productie_STO

In [17]:
# Laad dagelijks productie voor STO in STO dataframe
factory = 'STO'
dagelijkse_productie_STO = mdll.laad_dagelijkse_productie_in_dataframe(
                                factory, 
                                os.path.join(dagelijkse_productie_locatie, factory))
# dagelijkse_productie_STO

# voeg MSR waarden toe op het STO dagelijkse productie dataframe
dagelijkse_productie_STO = dagelijkse_productie_STO.merge(
                                        df_msr_gegevens, on='factory', how='left')
# dagelijkse_productie_STO

# verwijder records waar onderhoud werd gepland, zie opgave Deel 1, stap 5, bullet 1 en dixit Tim in de klas
dagelijkse_productie_STO = dagelijkse_productie_STO[dagelijkse_productie_STO['maintenance'] == 'No']
# dagelijkse_productie_STO

In [18]:
# Breng alle gegevens te samen in een groot dataframe, in geval van analyse totaal productie
dagelijkse_productie_ALL = pd.concat(
                            [dagelijkse_productie_BRU, 
                             dagelijkse_productie_STO],
                               ignore_index=True)
#  save het voor analyse in een Excel sheet
dagelijkse_productie_ALL.to_excel("../data/interim/dagelijkse_productie_ALL.xlsx")
# dagelijkse_productie_ALL

# D : een eerste controle van de gegevens

# E : de kolom 'productie' valideren en gebruiksklaar maken